In [1]:
from tools import load_yolo_model
import matplotlib.pyplot as plt
import supervision as sv


image 1/1 /workspaces/canprev-modules/IMG_0073.JPG: 544x416 1 botle_with_neckband, 1 label, 1 curved_shoulder, 1205.2ms
Speed: 62.4ms preprocess, 1205.2ms inference, 202.0ms postprocess per image at shape (1, 3, 544, 416)


In [4]:
detections = sv.Detections.from_ultralytics(result[0])

In [14]:
detections[detections.confidence > .80]

Detections(xyxy=array([[     785.34,        1269,      1368.1,      1916.5],
       [     805.53,      855.43,      1314.5,      1131.5]], dtype=float32), mask=None, confidence=array([    0.96136,     0.84135], dtype=float32), class_id=array([1, 0]), tracker_id=None, data={'class_name': array(['label', 'botle_with_neckband'], dtype='<U19')})

In [11]:
detections.data['class_name']

array(['label', 'botle_with_neckband', 'curved_shoulder'], dtype='<U19')

In [12]:
from collections import defaultdict

checklist = defaultdict(list)

def update_dict(key, value):
    checklist[key].append(value)

In [30]:
defaults = {
    'label': False, 
    'botle_with_neckband': False, 
    'curved_shoulder': False
}

In [35]:
for key, value in defaults.items():
    update_dict(key, value)
checklist

defaultdict(list,
            {'label': [False, False, False, False],
             'botle_with_neckband': [False, False, False, False],
             'curved_shoulder': [False, False, False, False]})

In [18]:
from collections import defaultdict

CHECKLIST = defaultdict(list)
THINGS_TO_CHECK = {'label', 'botle_with_neckband', 'curved_shoulder'}

def update_CHECKLIST(key, value):
    CHECKLIST[key].append(value)


def detect_side_view(image, view_name):
    global model
    # image_array = Image.open(image)

    result = model(source=image)[0]
    detections = sv.Detections.from_ultralytics(result)
    detections[detections.confidence > .60]
    for thing in THINGS_TO_CHECK:
        if thing in detections.data['class_name']:
            update_CHECKLIST(thing, True)

    return True

In [19]:
for side in ['IMG_0072.JPG', 'IMG_0073.JPG']:
    detect_side_view(image=side, view_name='side')


image 1/1 /workspaces/canprev-modules/IMG_0072.JPG: 544x416 1 botle_with_neckband, 1 label, 1 curved_shoulder, 189.5ms
Speed: 1.7ms preprocess, 189.5ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 416)

image 1/1 /workspaces/canprev-modules/IMG_0073.JPG: 544x416 1 botle_with_neckband, 1 label, 1 curved_shoulder, 218.2ms
Speed: 2.5ms preprocess, 218.2ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 416)


In [20]:
CHECKLIST

defaultdict(list,
            {'curved_shoulder': [True, True],
             'botle_with_neckband': [True, True],
             'label': [True, True]})

In [23]:
, , 

AttributeError: 'numpy.ndarray' object has no attribute 'upper'

In [ ]:
import streamlit as st
import numpy as np
from PIL import Image, ExifTags
from tools import load_combined_model, detect_object

# Load model
seal_model = load_combined_model(model_path='/home/prathijna_k/Documents/AI_canprev/update_class_label/finished_product.pt')

def correct_image_orientation(image):
    """Correct the orientation of an image based on its EXIF data."""
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break

        exif = image._getexif()
        if exif is not None:
            orientation = exif.get(orientation)
            if orientation == 3:
                image = image.rotate(180, expand=True)
            elif orientation == 6:
                image = image.rotate(270, expand=True)
            elif orientation == 8:
                image = image.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # Cases where the image doesn't have EXIF data
        pass
    return image

def main():
    st.title('Canprev AI')

    uploaded_file = st.file_uploader("Choose an image...", type=['jpg', 'jpeg', 'png'], key='Seal')

    if uploaded_file is not None:
        # Open and correct image orientation
        image = Image.open(uploaded_file)
        image = correct_image_orientation(image)
        image_np = np.array(image)
        
        # Create two columns
        col1, col2 = st.columns(2)

        with col1:
            st.header("Uploaded Image")
            st.image(image_np, caption='Uploaded Image', use_column_width=True)
        
        with col2:
            result_image = detect_object(image_np, model=seal_model)
            result_image_pil = Image.fromarray(result_image)

            st.header("Analysis")
            st.image(result_image_pil, caption='Analysis Result', use_column_width=True)

if _name_ == "__main__":
    main()
